# Synthetic Sonic Log Generation -- Add target for prediction

## <a>1. Imports</a>

In [1]:
# import libraries
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb
# import sklearn libararies
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression 
from sklearn.neural_network import MLPRegressor

## <a>2. Read Data</a>

In [2]:
df1 = pd.read_csv('train.csv')


In [3]:
df1.describe()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC,DTS
count,30143.000000,30143.000000,30143.000000,30143.000000,30143.000000,30143.000000,30143.000000,30143.000000,30143.000000
mean,-8.394576,-23.692615,38.959845,3.977690,1.547299,-17.446739,-20.229982,-54.891703,-9.737567
std,129.970219,157.142679,108.504554,365.112753,456.908969,149.083136,148.809506,372.858812,440.314119
min,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000
25%,8.058350,0.122800,17.248750,0.717700,0.712050,0.053100,2.226700,66.304350,118.534350
50%,8.625000,0.193600,36.821800,1.623000,1.628100,4.941500,2.432200,78.355100,137.689300
75%,9.048850,0.337150,58.346150,3.158300,3.280600,7.856650,2.551350,107.022500,182.973150
max,21.064200,3490.158200,1470.253400,10000.000000,60467.761700,28.106400,3.259700,155.980300,487.438400


In [4]:
# remove all rows that contains missing value
df1.replace(['-999', -999], np.nan, inplace=True)
df1.dropna(axis=0, inplace=True)
df1.describe()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC,DTS
count,20525.000000,20525.000000,20525.000000,20525.000000,20525.000000,20525.000000,20525.000000,20525.000000,20525.000000
mean,8.426679,0.274416,49.889253,2.598719,5.835466,3.833792,2.410734,88.312221,182.051067
std,1.845912,3.062495,54.811017,3.465665,422.449589,4.375818,0.181713,23.542419,84.670122
min,5.930400,0.014500,1.038900,0.123600,0.134100,-0.023200,0.680600,49.970500,80.580400
25%,6.629100,0.120300,16.036800,0.810000,0.797300,0.049800,2.236100,70.423100,127.148800
50%,8.578100,0.187700,37.498000,1.814900,1.829300,3.287800,2.466500,79.695400,142.678500
75%,8.671900,0.329000,61.140700,3.337400,3.463300,7.061300,2.563700,102.482800,192.757800
max,21.064200,365.885000,1470.253400,206.718200,60467.761700,28.106400,3.259700,155.980300,487.438400


# Scenario 1: Only use other Logs to predict DTS

In [5]:
#Only used Other logs to predict DTS
# seperate the features and targets
#df1_data = np.array(df1)
y_trainwell1 = df1.iloc[:,-1]
x_trainwell1 = df1.iloc[:,:-2]

# standize the matrix for training data
#scaler = StandardScaler()
#x_trainwell1 = scaler.fit_transform(x_trainwell1)

In [6]:
# train test split
# please remember to use random_state for all randomization steps, this will ensure we get the same results 
# as yours during the validation phase. 
X_train, X_test, y_train, y_test = train_test_split(x_trainwell1, y_trainwell1, test_size=0.2, random_state=100)

In [8]:
#Predict DTC only by using other logs, without DTS.
import sklearn.metrics as metrics
import xgboost as xgb
params={'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 17, 'n_estimators': 180, 'subsample': 0.9} 
xgbr = xgb.XGBRegressor(**params)
multi_xgb = xgbr.fit(X_train, y_train)
multi_xgb_pred = xgbr.predict(X_test)
RMSE=format(np.sqrt(metrics.mean_squared_error(y_test, multi_xgb_pred)),'.4f')   
print('RMSE:',  RMSE)

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:16:37] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 7.1792


# Scenario 2: Add DTC with other logs to predict DTS

In [27]:
#Add DTC to predict DTS
#df1_data = np.array(df1)
y_trainDTC = df1.iloc[:,-1]
x_trainDTC = df1.iloc[:,:-1]

x_trainDTC['DTC'] = df1['DTC']
x_trainDTC.head()
# standize the matrix for training data
#scaler = StandardScaler()
#x_trainwell1 = scaler.fit_transform(x_trainwell1)

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC
573,14.8824,0.5173,57.1903,1.9826,1.9726,27.9432,2.3510,125.9611
574,14.9570,0.4966,63.1593,2.0472,1.6373,28.1064,2.4055,125.9726
575,14.6341,0.4433,64.5217,2.0464,1.4866,26.5968,2.4579,125.8528
576,14.2318,0.4309,63.0144,2.0054,1.4613,19.5635,2.4219,125.7362
577,14.0163,0.4469,60.8421,1.9533,1.4438,9.8556,2.3138,125.9774


In [28]:
# train test split
# please remember to use random_state for all randomization steps, this will ensure we get the same results 
# as yours during the validation phase. 
X_trainDTC, X_testDTC, y_trainDTC, y_testDTC = train_test_split(x_trainDTC, y_trainDTC, test_size=0.2, random_state=100)

In [12]:
params={'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 17, 'n_estimators': 180, 'subsample': 0.9} 
xgbr = xgb.XGBRegressor(**params)
multi_xgb = xgbr.fit(X_trainDTC, y_trainDTC)
multi_xgb_pred = xgbr.predict(X_testDTC)
Rmse=format(np.sqrt(metrics.mean_squared_error(y_testDTC, multi_xgb_pred)),'.4f')   
print('RMSE:',  Rmse)


C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:21:38] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 5.3291


# Scenario 3: Use other logs to predict DTC

In [73]:
#Only used Other logs to predict DTS
# seperate the features and targets
#df1_data = np.array(df1)
y_trainwell1 = df1.iloc[:,-2]
x_trainwell1 = df1.iloc[:,:-2]

# standize the matrix for training data
#scaler = StandardScaler()
#x_trainwell1 = scaler.fit_transform(x_trainwell1)

In [74]:
# train test split
# please remember to use random_state for all randomization steps, this will ensure we get the same results 
# as yours during the validation phase. 
X_train, X_test, y_train, y_test = train_test_split(x_trainwell1, y_trainwell1, test_size=0.2, random_state=100)

In [39]:
params={'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 17, 'n_estimators': 180, 'subsample': 0.9} 
xgbr = xgb.XGBRegressor(**params)
multi_xgb = xgbr.fit(X_train, y_train)
multi_xgb_pred = xgbr.predict(X_test)
RMSE=format(np.sqrt(metrics.mean_squared_error(y_test, multi_xgb_pred)),'.4f')   
print('RMSE:',  RMSE)

[16:01:39] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 2.7313


# Scenario 4: Use DTS and other logs to predict DTC

In [16]:
#Add DTS to predict
# seperate the features and targets
#df1_data = np.array(df1)
y_trainDTS = df1.iloc[:,-2]
x_trainDTS = df1.iloc[:,:-2]

x_trainDTS['DTS'] = df1['DTS']
x_trainDTS.head()
# standize the matrix for training data
#scaler = StandardScaler()
#x_trainwell1 = scaler.fit_transform(x_trainwell1)

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTS
573,14.8824,0.5173,57.1903,1.9826,1.9726,27.9432,2.3510,306.8676
574,14.9570,0.4966,63.1593,2.0472,1.6373,28.1064,2.4055,305.8347
575,14.6341,0.4433,64.5217,2.0464,1.4866,26.5968,2.4579,305.9089
576,14.2318,0.4309,63.0144,2.0054,1.4613,19.5635,2.4219,305.1737
577,14.0163,0.4469,60.8421,1.9533,1.4438,9.8556,2.3138,304.4476


In [17]:
# train test split
# please remember to use random_state for all randomization steps, this will ensure we get the same results 
# as yours during the validation phase. 
X_trainDTS, X_testDTS, y_trainDTS, y_testDTS = train_test_split(x_trainDTS, y_trainDTS, test_size=0.2, random_state=100)

In [18]:

params={'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 17, 'n_estimators': 180, 'subsample': 0.9} 
xgbr = xgb.XGBRegressor(**params)
multi_xgb = xgbr.fit(X_trainDTS, y_trainDTS)
multi_xgb_pred = xgbr.predict(X_testDTS)
R2=format(np.sqrt(metrics.mean_squared_error(y_testDTS, multi_xgb_pred)),'.4f')   
print('RMSE:',  R2)

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:32:24] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 1.7923


# Use Pseudolable to predict lost DTC (other logs + DTS)
Add DTC to the original data, then predict DTC with more training data, see if DTS is improved with new DTC training data

In [21]:
df_original = pd.read_csv('train.csv')
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30143 entries, 0 to 30142
Data columns (total 9 columns):
CAL     30143 non-null float64
CNC     30143 non-null float64
GR      30143 non-null float64
HRD     30143 non-null float64
HRM     30143 non-null float64
PE      30143 non-null float64
ZDEN    30143 non-null float64
DTC     30143 non-null float64
DTS     30143 non-null float64
dtypes: float64(9)
memory usage: 2.1 MB


In [47]:
#remove all rows that contains missing value
df_original['DTC'].replace(['-999', -999], np.nan, inplace=True)
df_original['DTS'].replace(['-999', -999], np.nan, inplace=True)
df_original['CAL'].replace(['-999', -999], np.nan, inplace=True)
df_original['CNC'].replace(['-999', -999], np.nan, inplace=True)
df_original['PE'].replace(['-999', -999], np.nan, inplace=True)
df_original['ZDEN'].replace(['-999', -999], np.nan, inplace=True)
df_original['HRD'].replace(['-999', -999], np.nan, inplace=True)
df_original['HRM'].replace(['-999', -999], np.nan, inplace=True)

In [48]:
DTC_test = df_original[(df_original['DTC'].isnull()) & (df_original['DTS'].notnull())] 

In [50]:
DTC_test

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC,DTS
9071,9.2283,0.1511,19.6345,2.0004,2.0025,4.9002,2.4999,NaN,154.5933
9072,9.2053,0.1462,18.0560,2.0411,2.0438,5.0943,2.5163,NaN,154.2602
9073,9.1987,0.1500,16.7997,2.0366,2.0418,4.8517,2.4698,NaN,153.7306
9074,9.2158,0.1484,16.3179,1.9926,2.0000,4.7805,2.5137,NaN,153.1662
9075,9.2219,0.1549,14.8773,1.9588,1.9652,4.7819,2.4858,NaN,152.5901
...,...,...,...,...,...,...,...,...,...
13040,NaN,NaN,-999.0000,1.1678,1.1479,NaN,NaN,NaN,144.7557
13041,NaN,NaN,-999.0000,1.2992,1.2661,NaN,NaN,NaN,143.5288
13042,NaN,NaN,-999.0000,1.4075,1.3666,NaN,NaN,NaN,142.8309
13043,NaN,NaN,-999.0000,1.4212,1.3870,NaN,NaN,NaN,142.6761


In [55]:
#Splitting DTC_test properly
y_testmiss = DTC_test.iloc[:,-2]
x_testmiss = DTC_test.iloc[:,:-2]

x_testmiss['DTS'] = DTC_test['DTS']
x_testmiss.head()

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTS
9071,9.2283,0.1511,19.6345,2.0004,2.0025,4.9002,2.4999,154.5933
9072,9.2053,0.1462,18.0560,2.0411,2.0438,5.0943,2.5163,154.2602
9073,9.1987,0.1500,16.7997,2.0366,2.0418,4.8517,2.4698,153.7306
9074,9.2158,0.1484,16.3179,1.9926,2.0000,4.7805,2.5137,153.1662
9075,9.2219,0.1549,14.8773,1.9588,1.9652,4.7819,2.4858,152.5901


In [51]:
#Add DTS to predict
# seperate the features and targets
#df1_data = np.array(df1)
y_trainDTS = df1.iloc[:,-2]
x_trainDTS = df1.iloc[:,:-2]

x_trainDTS['DTS'] = df1['DTS']
x_trainDTS.head()
# standize the matrix for training data
#scaler = StandardScaler()
#x_trainwell1 = scaler.fit_transform(x_trainwell1)

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTS
573,14.8824,0.5173,57.1903,1.9826,1.9726,27.9432,2.3510,306.8676
574,14.9570,0.4966,63.1593,2.0472,1.6373,28.1064,2.4055,305.8347
575,14.6341,0.4433,64.5217,2.0464,1.4866,26.5968,2.4579,305.9089
576,14.2318,0.4309,63.0144,2.0054,1.4613,19.5635,2.4219,305.1737
577,14.0163,0.4469,60.8421,1.9533,1.4438,9.8556,2.3138,304.4476


In [52]:
# train test split
# please remember to use random_state for all randomization steps, this will ensure we get the same results 
# as yours during the validation phase. 
X_traindts, X_testdts, y_traindts, y_testdts = train_test_split(x_trainDTS, y_trainDTS, test_size=0.2, random_state=100)

In [54]:
params={'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 17, 'n_estimators': 180, 'subsample': 0.9} 
xgbr = xgb.XGBRegressor(**params)
multi_xgb = xgbr.fit(X_traindts, y_traindts)
multi_xgb_pred = xgbr.predict(X_testdts)
R2=format(np.sqrt(metrics.mean_squared_error(y_testdts, multi_xgb_pred)),'.4f')   
print('RMSE:',  R2)

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[16:59:52] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 1.7923


In [56]:
#******Pseudolableing class************************
from sklearn.utils import shuffle
from sklearn.base import BaseEstimator, RegressorMixin

class PseudoLabeler(BaseEstimator, RegressorMixin):
    def __init__(self, model, unlabled_data, features, target, sample_rate=0.2, seed=42):
        assert sample_rate <= 1.0, 'Sample_rate should be between 0.0 and 1.0.'
        self.sample_rate = sample_rate
        self.seed = seed
        self.model = model
        self.model.seed = seed

        self.unlabled_data = unlabled_data
        self.features = features
        self.target = target
    
    def get_params(self, deep=True):
        return {"sample_rate": self.sample_rate,"seed": self.seed,"model": self.model,"unlabled_data": self.unlabled_data,"features": self.features,"target": self.target}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def fit(self, X, y):
        augemented_train = self.__create_augmented_train(X, y)
        self.model.fit(augemented_train[self.features],augemented_train[self.target])
        return self
    
    def __create_augmented_train(self, X, y):
        num_of_samples = int(len(self.unlabled_data) * self.sample_rate)
        self.model.fit(X, y)
        pseudo_labels = self.model.predict(self.unlabled_data[self.features])
        pseudo_data = self.unlabled_data.copy(deep=True)
        pseudo_data[self.target] = pseudo_labels
        # Take a subset of the test set with pseudo-labels and append in onto
        # the training set
        sampled_pseudo_data = pseudo_data.sample(n=num_of_samples)
        temp_train = pd.concat([X, y], axis=1)
        augemented_train = pd.concat([sampled_pseudo_data, temp_train])
        return shuffle(augemented_train)
    
    def predict(self, X):
        return self.model.predict(X)
    def get_model_name(self):
        return self.model.__class__.__name__

# Keep changing sample rate, to inject predicted DTC data into training. 
## If there is no improvement in CV score, then there is no improvement for adding more predicted DTC in training. 

# Another word, no need to fill in missing target in this competition

In [61]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
features = x_testmiss.columns
target = 'DTC'

# fitting
params={'objective':'reg:squarederror', 'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 20, 'n_estimators': 180, 'subsample': 0.9}
xgbr = xgb.XGBRegressor(**params)

sample_rates = np.linspace(0.1, 1, 10)
for sample_rate in sample_rates:
    pseudo_label = PseudoLabeler(xgbr,x_testmiss,features,target,sample_rate)
    pseudo_label.seed = 42
    pseudo_label.fit(X_traindts, y_traindts)
    cv_results = model_selection.cross_val_score(pseudo_label, X_traindts, y_traindts, scoring='neg_root_mean_squared_error', cv=5)
    print(sample_rate)
    print('Best score: {}'.format(cv_results.max()))
    print('Mean score: {}'.format(cv_results.mean()))
    print('std score: {}'.format(cv_results.std()))

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

0.1
Best score: -1.7576246192177176
Mean score: -1.8839541387609615
std score: 0.09986256567924083


C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

0.2
Best score: -1.750326828512304
Mean score: -1.8830961750888882
std score: 0.10302286839908655


C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

...

0.8
Best score: -1.7767112709782484
Mean score: -1.8781989236228989
std score: 0.10012582150629216


C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

0.9
Best score: -1.7641464111740617
Mean score: -1.8688112433202424
std score: 0.10017108434038194


C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

1.0
Best score: -1.771209151542699
Mean score: -1.8775637698915233
std score: 0.09709207545358617


In [62]:
features = x_testmiss.columns
target = 'DTC'
# fitting
params={'objective':'reg:squarederror', 'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 20, 'n_estimators': 180, 'subsample': 0.9}
xgbr = xgb.XGBRegressor(**params)

sample_rates = 0.9
pseudo_label = PseudoLabeler(xgbr,x_testmiss,features,target,sample_rate)
pseudo_label.seed = 42
pseudo = pseudo_label.fit(X_traindts, y_traindts)

pseudo_pred = pseudo.predict(X_testdts)
RMSE=format(np.sqrt(metrics.mean_squared_error(y_testdts, pseudo_pred)),'.4f')   
print('RMSE:',  RMSE)

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 1.7846


In [64]:
pseudo_pred1 = pseudo.predict(x_testmiss)
pseudo_pred1

array([79.063545, 79.915794, 80.427826, ..., 68.453415, 68.453415,
       68.453415], dtype=float32)

In [65]:
x_testmiss['DTC'] = pseudo_pred1
x_testmiss

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTS,DTC
9071,9.2283,0.1511,19.6345,2.0004,2.0025,4.9002,2.4999,154.5933,79.063545
9072,9.2053,0.1462,18.0560,2.0411,2.0438,5.0943,2.5163,154.2602,79.915794
9073,9.1987,0.1500,16.7997,2.0366,2.0418,4.8517,2.4698,153.7306,80.427826
9074,9.2158,0.1484,16.3179,1.9926,2.0000,4.7805,2.5137,153.1662,79.401039
9075,9.2219,0.1549,14.8773,1.9588,1.9652,4.7819,2.4858,152.5901,80.336815
...,...,...,...,...,...,...,...,...,...
13040,NaN,NaN,-999.0000,1.1678,1.1479,NaN,NaN,144.7557,69.020950
13041,NaN,NaN,-999.0000,1.2992,1.2661,NaN,NaN,143.5288,68.727249
13042,NaN,NaN,-999.0000,1.4075,1.3666,NaN,NaN,142.8309,68.453415
13043,NaN,NaN,-999.0000,1.4212,1.3870,NaN,NaN,142.6761,68.453415


# Add the predicted missing DTC portion back to df1 for training.

In [78]:
#Only used Other logs to predict DTS
# seperate the features and targets
#df1_data = np.array(df1)
y_trainwell1 = df1.iloc[:,-2]
x_trainwell1 = df1.iloc[:,:-2]

# standize the matrix for training data
#scaler = StandardScaler()
#x_trainwell1 = scaler.fit_transform(x_trainwell1)

In [79]:
X_train, X_test, y_train, y_test = train_test_split(x_trainwell1, y_trainwell1, test_size=0.2, random_state=100)

In [80]:
y_train_full = pd.concat([y_train,x_testmiss['DTC']])
y_train_full

21267    96.734700
24769    71.280500
19403    80.812000
25648    72.254300
19710    94.216900
           ...    
13040    69.020950
13041    68.727249
13042    68.453415
13043    68.453415
13044    68.453415
Name: DTC, Length: 20394, dtype: float64

In [81]:
x_train_full = pd.concat([X_train,x_testmiss.iloc[:,:-2]])
x_train_full

,CAL,CNC,GR,HRD,HRM,PE,ZDEN
21267,8.5469,0.2614,31.2806,0.3275,0.3266,0.0564,2.1930
24769,8.6250,0.0941,20.8876,3.2605,3.4541,0.0469,2.5802
19403,5.9882,0.2119,18.1301,6.0357,5.2180,4.5300,2.1301
25648,8.6250,0.1225,37.1426,4.3130,4.5483,0.0559,2.6122
19710,7.5276,0.3004,88.2008,1.5794,1.6035,10.7551,2.4215
...,...,...,...,...,...,...,...
13040,NaN,NaN,-999.0000,1.1678,1.1479,NaN,NaN
13041,NaN,NaN,-999.0000,1.2992,1.2661,NaN,NaN
13042,NaN,NaN,-999.0000,1.4075,1.3666,NaN,NaN
13043,NaN,NaN,-999.0000,1.4212,1.3870,NaN,NaN


In [82]:
params={'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 17, 'n_estimators': 180, 'subsample': 0.9} 
xgbr = xgb.XGBRegressor(**params)
multi_xgb = xgbr.fit(x_train_full, y_train_full)
multi_xgb_pred = xgbr.predict(X_test)
RMSE=format(np.sqrt(metrics.mean_squared_error(y_test, multi_xgb_pred)),'.4f')   
print('RMSE:',  RMSE)

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[20:21:19] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 2.6637


# Final result improved by 2.5% from 2.73 to 2.66. It is very important only pseudolabel on train data. Do not contaminate your test data!

# Use Pseudolable to predict lost DTS (other logs + DTC)

In [83]:
DTS_test = df_original[(df_original['DTS'].isnull()) & (df_original['DTC'].notnull())] 
DTS_test

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC,DTS
4114,12.0049,0.5950,33.5426,0.4952,0.4876,15.7609,2.2328,136.4907,NaN
4115,12.2566,0.6582,34.8843,0.5009,0.4921,17.8313,2.1860,136.7395,NaN
4116,14.0826,0.7457,36.4860,0.5097,0.5035,17.3974,2.1061,136.1385,NaN
4117,14.6995,0.8387,33.1176,0.5208,0.5208,15.1210,2.1138,134.1399,NaN
4118,11.8761,0.6948,31.2897,0.5321,0.5371,15.5359,2.1827,131.5483,NaN
...,...,...,...,...,...,...,...,...,...
27978,8.6250,0.3223,133.2228,1.1364,0.8143,0.0794,2.5593,87.0442,NaN
27979,8.6484,0.3308,129.8646,0.9289,0.5869,0.0699,2.5276,86.6879,NaN
27980,8.6250,0.3347,129.2383,1.0534,0.8142,0.0640,2.5065,86.4585,NaN
27981,8.6250,0.3376,131.2858,1.4505,1.2257,0.0593,2.4904,86.3668,NaN


In [84]:
#Splitting DTC_test properly
y_testmiss = DTS_test.iloc[:,-1]
x_testmiss = DTS_test.iloc[:,:-1]

x_testmiss.head()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC
4114,12.0049,0.5950,33.5426,0.4952,0.4876,15.7609,2.2328,136.4907
4115,12.2566,0.6582,34.8843,0.5009,0.4921,17.8313,2.1860,136.7395
4116,14.0826,0.7457,36.4860,0.5097,0.5035,17.3974,2.1061,136.1385
4117,14.6995,0.8387,33.1176,0.5208,0.5208,15.1210,2.1138,134.1399
4118,11.8761,0.6948,31.2897,0.5321,0.5371,15.5359,2.1827,131.5483


In [85]:
#Add DTS to predict
# seperate the features and targets
#df1_data = np.array(df1)
y_trainDTC = df1.iloc[:,-1]
x_trainDTC = df1.iloc[:,:-1]

x_trainDTC.head()

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC
573,14.8824,0.5173,57.1903,1.9826,1.9726,27.9432,2.3510,125.9611
574,14.9570,0.4966,63.1593,2.0472,1.6373,28.1064,2.4055,125.9726
575,14.6341,0.4433,64.5217,2.0464,1.4866,26.5968,2.4579,125.8528
576,14.2318,0.4309,63.0144,2.0054,1.4613,19.5635,2.4219,125.7362
577,14.0163,0.4469,60.8421,1.9533,1.4438,9.8556,2.3138,125.9774


In [86]:
# train test split
# please remember to use random_state for all randomization steps, this will ensure we get the same results 
# as yours during the validation phase. 
X_traindtc, X_testdtc, y_traindtc, y_testdtc = train_test_split(x_trainDTC, y_trainDTC, test_size=0.2, random_state=100)

In [87]:
params={'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 17, 'n_estimators': 180, 'subsample': 0.9} 
xgbr = xgb.XGBRegressor(**params)
multi_xgb = xgbr.fit(X_traindtc, y_traindtc)
multi_xgb_pred = xgbr.predict(X_testdtc)
R2=format(np.sqrt(metrics.mean_squared_error(y_testdtc, multi_xgb_pred)),'.4f')   
print('RMSE:',  R2)

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[20:32:21] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 5.3291


In [88]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
features = x_testmiss.columns
target = 'DTS'

# fitting
params={'objective':'reg:squarederror', 'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 20, 'n_estimators': 180, 'subsample': 0.9}
xgbr = xgb.XGBRegressor(**params)

sample_rates = np.linspace(0.1, 1, 10)
for sample_rate in sample_rates:
    pseudo_label = PseudoLabeler(xgbr,x_testmiss,features,target,sample_rate)
    pseudo_label.seed = 42
    pseudo_label.fit(X_traindtc, y_traindtc)
    cv_results = model_selection.cross_val_score(pseudo_label, X_traindtc, y_traindtc, scoring='neg_root_mean_squared_error', cv=5)
    print(sample_rate)
    print('Best score: {}'.format(cv_results.max()))
    print('Mean score: {}'.format(cv_results.mean()))
    print('std score: {}'.format(cv_results.std()))

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

0.1
Best score: -5.987950633912173
Mean score: -6.2625619922938025
std score: 0.31047773092460573


C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

0.2
Best score: -6.083729349030752
Mean score: -6.302748182299373
std score: 0.21698842484540629


C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

...

0.8
Best score: -6.0842415823776586
Mean score: -6.285011248503927
std score: 0.19078388402376573


C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

0.9
Best score: -5.958702773133646
Mean score: -6.269986475885145
std score: 0.23196101424029833


C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

1.0
Best score: -6.022468110012249
Mean score: -6.311639595357078
std score: 0.2115932911320369


In [89]:
features = x_testmiss.columns
target = 'DTS'
# fitting
params={'objective':'reg:squarederror', 'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 20, 'n_estimators': 180, 'subsample': 0.9}
xgbr = xgb.XGBRegressor(**params)

sample_rates = 0.9
pseudo_label = PseudoLabeler(xgbr,x_testmiss,features,target,sample_rate)
pseudo_label.seed = 42
pseudo = pseudo_label.fit(X_traindtc, y_traindtc)

pseudo_pred = pseudo.predict(X_testdtc)
RMSE=format(np.sqrt(metrics.mean_squared_error(y_testdtc, pseudo_pred)),'.4f')   
print('RMSE:',  RMSE)

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


RMSE: 5.3263


In [90]:
pseudo_pred1 = pseudo.predict(x_testmiss)
pseudo_pred1

array([443.563  , 412.12744, 427.06064, ..., 177.7156 , 175.4813 ,
       179.1106 ], dtype=float32)

In [91]:
x_testmiss['DTS'] = pseudo_pred1
x_testmiss

,CAL,CNC,GR,HRD,HRM,PE,ZDEN,DTC,DTS
4114,12.0049,0.5950,33.5426,0.4952,0.4876,15.7609,2.2328,136.4907,443.562988
4115,12.2566,0.6582,34.8843,0.5009,0.4921,17.8313,2.1860,136.7395,412.127441
4116,14.0826,0.7457,36.4860,0.5097,0.5035,17.3974,2.1061,136.1385,427.060638
4117,14.6995,0.8387,33.1176,0.5208,0.5208,15.1210,2.1138,134.1399,433.769470
4118,11.8761,0.6948,31.2897,0.5321,0.5371,15.5359,2.1827,131.5483,405.608032
...,...,...,...,...,...,...,...,...,...
27978,8.6250,0.3223,133.2228,1.1364,0.8143,0.0794,2.5593,87.0442,173.698593
27979,8.6484,0.3308,129.8646,0.9289,0.5869,0.0699,2.5276,86.6879,170.271042
27980,8.6250,0.3347,129.2383,1.0534,0.8142,0.0640,2.5065,86.4585,177.715607
27981,8.6250,0.3376,131.2858,1.4505,1.2257,0.0593,2.4904,86.3668,175.481293


# Add the predicted missing DTS portion back to df1 for training.

In [93]:
#dforigin_train = np.array(df_original_train)
y_trainwell1 = df1.iloc[:,-1]
x_trainwell1 = df1.iloc[:,:-2]

In [94]:
X_train, X_test, y_train, y_test = train_test_split(x_trainwell1, y_trainwell1, test_size=0.2, random_state=100)

In [97]:
y_train_full = pd.concat([y_train,x_testmiss['DTS']])
y_train_full

21267    180.807800
24769    135.738000
19403    179.458400
25648    133.844500
19710    203.530700
            ...    
27978    173.698593
27979    170.271042
27980    177.715607
27981    175.481293
27982    179.110596
Name: DTS, Length: 21205, dtype: float64

In [98]:
x_train_full = pd.concat([X_train,x_testmiss.iloc[:,:-2]])
x_train_full

,CAL,CNC,GR,HRD,HRM,PE,ZDEN
21267,8.5469,0.2614,31.2806,0.3275,0.3266,0.0564,2.1930
24769,8.6250,0.0941,20.8876,3.2605,3.4541,0.0469,2.5802
19403,5.9882,0.2119,18.1301,6.0357,5.2180,4.5300,2.1301
25648,8.6250,0.1225,37.1426,4.3130,4.5483,0.0559,2.6122
19710,7.5276,0.3004,88.2008,1.5794,1.6035,10.7551,2.4215
...,...,...,...,...,...,...,...
27978,8.6250,0.3223,133.2228,1.1364,0.8143,0.0794,2.5593
27979,8.6484,0.3308,129.8646,0.9289,0.5869,0.0699,2.5276
27980,8.6250,0.3347,129.2383,1.0534,0.8142,0.0640,2.5065
27981,8.6250,0.3376,131.2858,1.4505,1.2257,0.0593,2.4904


In [99]:
params={'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 17, 'n_estimators': 180, 'subsample': 0.9} 
xgbr = xgb.XGBRegressor(**params)
multi_xgb = xgbr.fit(x_train_full, y_train_full)
multi_xgb_pred = xgbr.predict(X_test)
RMSE=format(np.sqrt(metrics.mean_squared_error(y_test, multi_xgb_pred)),'.4f')   
print('RMSE:',  RMSE)

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:50:48] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE: 7.6630


# The RMSE after Psudolabel is worse than before. So It does not help.

Tried to apply Psudolabel method to test data, and then see if the results become much worse. Then it is a indication of bad prediction?

In [100]:
df_test = pd.read_csv('test.csv')
df_test

,CAL,CNC,GR,HRD,HRM,PE,ZDEN
0,8.5781,0.3521,55.1824,0.8121,0.7810,6.8291,2.3256
1,8.5781,0.3639,57.0114,0.8038,0.7723,6.8100,2.3255
2,8.5781,0.3703,58.9263,0.7444,0.7048,6.7766,2.3212
3,8.5625,0.3667,57.3308,0.7169,0.6542,6.7219,2.3119
4,8.5781,0.3500,53.0624,0.6845,0.6109,6.6384,2.2982
...,...,...,...,...,...,...,...
11083,8.7813,0.1882,68.0333,1.0280,0.9815,6.5003,2.4423
11084,8.7344,0.1879,68.0753,1.0249,0.9822,6.4415,2.4377
11085,8.7266,0.1824,68.8516,1.0459,1.0199,6.3413,2.4342
11086,8.7500,0.1822,70.8515,1.0609,1.0541,6.2254,2.4320


In [102]:
params={'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 17, 'n_estimators': 180, 'subsample': 0.9} 
xgbr = xgb.XGBRegressor(**params)
multi_xgb = xgbr.fit(x_trainwell1, y_trainwell1)
multi_xgb_pred = xgbr.predict(df_test)
multi_xgb_pred

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[22:33:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([205.47127, 204.32239, 200.61469, ..., 139.47382, 147.15788,
       148.07326], dtype=float32)

In [103]:
len(multi_xgb_pred)

11088

In [113]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
features = df_test.columns
target = 'DTS'

# fitting
params={'objective':'reg:squarederror', 'colsample_bytree': 0.9, 'gamma': 0.6, 'learning_rate': 0.05, 'max_depth': 20, 'n_estimators': 180, 'subsample': 0.9}
xgbr = xgb.XGBRegressor(**params)

sample_rates = np.linspace(0.1, 1, 10)
for sample_rate in sample_rates:
    pseudo_label = PseudoLabeler(xgbr,df_test,features,target,sample_rate)
    pseudo_label.seed = 42
    pseudo_label.fit(X_train, y_train)
    cv_results = model_selection.cross_val_score(pseudo_label, X_train, y_train, scoring='neg_root_mean_squared_error', cv=5)
    print(sample_rate)
    print('Best score: {}'.format(cv_results.max()))
    print('Mean score: {}'.format(cv_results.mean()))
    print('std score: {}'.format(cv_results.std()))

C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

0.1
Best score: -7.762378892521971
Mean score: -8.024213692577048
std score: 0.23324537160881126


C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

0.2
Best score: -7.768904622104352
Mean score: -8.031720095637219
std score: 0.22806670822044248


C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

...

0.8
Best score: -7.767193586269825
Mean score: -8.09406465931717
std score: 0.2474042727781612


C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

0.9
Best score: -7.846415826629241
Mean score: -8.102742482077655
std score: 0.18434448665289369


C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
C:\Users\bohu8515\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: F

1.0
Best score: -7.88609527807074
Mean score: -8.074823281320397
std score: 0.20028580969290397
